### System rekomendujący na bazie Netflix Prize award

1. Zacznijmy od obejrzenia wykładu [rozdział 9](http://www.mmds.org/)
2. proszę ściągnac bazę netflixa o której była mowa w/w wykładzie [z kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) (2GB) 

Zaczniemy od wczytania danych, ponizszy listing wczytuje dane z jednego pliku i robi z nich trójkę (user, product, rating). Wykorzystamy do tego predefiniowany obiekt Rating z mlib.recommendation (prosze zwrócic uwagę że w tej konwencji nasz film będzie produktem)

#### Zadanie 1:
zmodyfikuj poniższy listing tak aby wczytywać wsystkie pięc plików

In [1]:
import pyspark
from pyspark.mllib.recommendation import Rating
from pyspark import SparkContext

# tutaj zaincjalizujemy klaster (na jednym komputerze) sparka
# zmodyfikujemy nieco std ustawienia maszyny java zwiększając domyslne (bardzo małe) limity pamieci
# local[*] oznacza że użyjemy wsystkich rdzeni - jeśli zabrankie nam ramu możemy zmniejszyć ilość tą ilość
# polecam spoglądać do konsoli na http://localhost:4040/ aby monitorować zużycie zasobów

conf = pyspark.SparkConf().setAppName("recommendation")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '20G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

files = ['./RS/combined_data_1.txt',
        './RS/combined_data_2.txt',
        './RS/combined_data_3.txt',
        './RS/combined_data_4.txt'] # Ufam, że użytkownik wprowadził nazwy plików, które istnieją...

ratings = []
for single_name in files:
    f = open(single_name)
    for i, line in enumerate(f):
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line[:-1])        
        else:
            user_id, rating, _ = line.split(',')
            r = Rating(int(user_id), int(movie_id), int(rating))
            ratings.append(r)
    f.close()
    print("File '" + single_name + "' loaded!")

print('Finished! Processed lines: ' + str(len(ratings)))

File './RS/combined_data_1.txt' loaded!
File './RS/combined_data_2.txt' loaded!
File './RS/combined_data_3.txt' loaded!
File './RS/combined_data_4.txt' loaded!
Finished!


In [5]:
#tutaj wczytamy identyfikatory filmów i ich tytuły

f = open('./RS/movie_titles.csv', encoding = "ISO-8859-1")
g = [l.strip().split(',') for l in f.readlines()]
id2title = {int(a[0]):','.join(a[2:]) for a in g}
f.close()
print('Finished!')

Finished!


In [6]:
id2title
print('Finished!')

Finished!


In [7]:
# utworzmy z naszej listy ratingow zasob rdd (rozproszenie)
ratings_rdd = sc.parallelize(ratings)
print('Finished!')

Finished!


#### Zadanie 2
Przefiltruj ratings_rdd aby wziac pod uwage filmy ktory maja co najmniej 50 ocen
tip: zrób napierw liste par (movie_id, Rating) i pogrupuj ją przy użyciu GroupByKey a nstępnie przefiltruj

In [8]:
ratings_pairs = ratings_rdd.map(lambda r: (r.product, r))
rg = ratings_keyed.groupByKey().  ????

??????
ratings_filtered = ???????.collect()

PythonRDD[1] at RDD at PythonRDD.scala:53


In [5]:
# zasob rdd na przefiltrowanym obiekcie
ratings_filtered = sc.parallelize(ratings_filtered)

Dokonamy teraz faktoryzacji macierzy, nasej utility Matrix (zasób RDD ratings_filtered jest własnie taką macierzą) przy użyciu aproksymacji algorytmu spadku gradientowego
![alt](https://edersoncorbari.github.io/assets/images/blog/als-matrix-rec-calc.png)

In [245]:
rank = 10
numIterations = 10
model = ALS.train(ratings_filtered, rank, numIterations)

Zrzucilismy uzytkowników i filmy na nowy wymiar o wielkości 10. Możemy wprost poprosić o te macierze

In [246]:
users  = model.userFeatures()
movies = model.productFeatures()

Gęsta ale 'wąska' macierz movies zmieści się nam już bezproblemowo w RAM, a zatem zróbmy z niej po prostu macierz numpy

In [247]:
movies_mtx = np.array(movies.map(lambda rv:rv[1]).collect())
movie2row_number = movies.map(lambda rv:rv[0]).collect() #movie id to row number

Zróbmy prosty ekesperyment, zobaczmy czy jakie filmy są 'podobne' do 'Władcy pierścieni'

In [115]:
id2title[1757] # 'The Lord of the Rings'

lotr_idx = movie2row_number.index(1757)
lotr_vector = movies_mtx[lotr_idx]
lotr_vector #to jest wladca pierscienie rzutowany na latent space

array([ 0.29456642, -0.24367087, -0.11916542, -0.70450312, -0.52015817,
        0.22564895,  0.08156089,  0.19098088, -0.75671792,  0.3677029 ])

In [251]:
# obliczmy macierz odleglosci 
from scipy.spatial import distance
ds = distance.cdist([lotr_vector], m_l, 'cosine')[0]
ds.argsort()[:10]

array([2018,  928, 3172, 3063,  605,  206, 1237, 3116,  734, 1296])

In [253]:
print(ds[2018], ':', id2title[movie2row_number[2018]])
print(ds[928], ':', id2title[movie2row_number[928]])

0.0 : The Lord of the Rings
0.05174156358429072 : The Return of the King


Odległosć 0.0 to oczywiscie ten sam obiekt a zatem interesuje nas drugi w kolejności wpis. I cóż za niespodzianka? ludzie którzy ocenili wysoko Władce pierścieni ocenili również wysokos Powrót Króla

Na ile cech powinniśmy faktoryzować Utility Matrix? 

#### Zadanie 3
Dobierz paramter rank (ilość cech) dla obiektu ALS dzieląc zbiór ratings_filtered na 80% vs 20% (treningowy i testowy, tak jak na wykłdzie). Następnie wykorzystać metodę ALS.recommendProductsForUsers aby polecić filmy i zweryfikuj ile filmów udało Ci sie prawidłowo polecić ze zbioru testowego. Odpowiednio modyfikuj parametr rank aby otrzymać względnie wysoki wynik rekomendacji jednocześnie utrymując w miare wąskie macierze

#### Zadanie 4
Zrób program "co obejrzeć dziś wieczorem".
Program powinien pokazaywać 5 tytułow, uzytkownik wskazuje jeden lub kilka z nich i system pokazuje kolejne rekomendacje. Każda kolejna rekomendacja powinna wskazywać coraz lepsze rekomendacje. 

problem można rozwiazać na kilka sposób. Można pogrupować przestrzeń filmów (macierz P.T) i pokazywać z różnych np. centroidy klastrów (które będą reprezentować gatunki filmów (prawdopodobnie) ). 

każde kolejne wybranie filmu będzie wymagało stworzenie nowego sztuznego użytkownika z ratingiem. W każdej iteracji takiego użytkownika należy rzucic na macierz Q (czyli na latentną macierz użytkownika). Przypatrz się dokładnie sposobowi mnożenia na obrazku powyżej i zastanów się jak otrzymać takiego użytkownika w przestrzeni cech?). Przypominam że macierz P.T jest nieodracalna ale można wykorzystać pseudo odwrotnosć.